In [1]:
%load_ext autoreload
%autoreload 2

from utils import *
import numpy as np
import pandas as pd
import sys
sys.path.append("..")

from torch.utils import data

import glob
import random
import os
import json
import time
import matplotlib.pyplot as plt
import seaborn as sns


from bokeh.io import push_notebook, show, output_notebook,reset_output
from bokeh.plotting import figure
import networkx as nx
from bokeh.plotting import figure, from_networkx, curdoc,row,column
from bokeh.models import Rect, HoverTool,  TextInput, GraphRenderer, Circle
from bokeh.io import output_file
from bokeh.plotting import figure, output_file, save
from bokeh.models import (BoxZoomTool, Circle, Line, HoverTool,
                          MultiLine, Plot, Range1d, ResetTool)
from bokeh.models import LinearColorMapper, BasicTicker, ColorBar,CustomJSHover,CustomJS
from bokeh.models import Ellipse, GraphRenderer, StaticLayoutProvider
from bokeh.models import Select, DataRange1d, ColumnDataSource, MultiChoice


import grandalf

output_notebook()

%load_ext autoreload
%autoreload 2

Loading BokehJS ...

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
QDIR = os.path.join("/Users/pari/LatencyCollector/queries")
JOB_QNAMES = get_qnames(os.path.join(QDIR, "job"))
CEB_QNAMES = get_qnames(os.path.join(QDIR, "ceb-small"))

MULTIPLE=False
EXP_NAME = "CEB-t2medium-gp2"
#EXP_NAME = "CEB-t2small-gp3"
#EXP_NAME = "CEB-t2medium-gp3"
#EXP_NAME = "m5ad-gp2"
#EXP_NAME = "CEB-c7large-gp2"

# ## w/ 16 processes
#EXP_NAME = "t7xlarge-gp3-c"

## w/ 9 processes
#EXP_NAME = "t7xlarge-gp3-e-stressng"
#EXP_NAMES = ["t7xlarge-gp3-c", "t7xlarge-gp3-d", "t7xlarge-gp3-e-stressng"]

#EXP_NAME = "t7xlarge-gp3-d"

if MULTIPLE:
    RESULTDIR = os.path.join("/Users/pari/prism-testbed/", "LatencyCollectorResults", "multiple", EXP_NAME)
else:
    RESULTDIR = os.path.join("/Users/pari/prism-testbed/", "LatencyCollectorResults", EXP_NAME)

#RESULTDIR = os.path.join("/Users/pari/prism-testbed/", "LatencyCollectorResults")
#print(RESULTDIR)
#CEB-t2medium-gp2

In [3]:
# print(os.path.exists(RESULTDIR))

dfs = []
# for exp in EXP_NAMES:
#     resultdir = os.path.join("/Users/pari/prism-testbed/", "LatencyCollectorResults", "multiple", exp)

print(RESULTDIR)
assert os.path.exists(RESULTDIR)

rtfns = glob.iglob(RESULTDIR + "/*/results/Runtime*.csv")
for rtfn in rtfns:
    print(rtfn)
    dfs.append(pd.read_csv(rtfn))
    
df = pd.concat(dfs)

/Users/pari/prism-testbed/LatencyCollectorResults/CEB-t2medium-gp2
/Users/pari/prism-testbed/LatencyCollectorResults/CEB-t2medium-gp2/i-0110d5685ad7c0dfe/results/Runtimes.csv


In [4]:
print(df["runtime"].sum())
print(df["runtime"].describe())

99854.79351329803
count    595.000000
mean     167.823182
std      289.108832
min        0.005181
25%        3.091356
50%       17.632031
75%      171.315723
max      909.000000
Name: runtime, dtype: float64


In [5]:
qworkloads = {}
for qname in JOB_QNAMES:
    qworkloads[qname] = "JOB"
for qname in CEB_QNAMES:
    qworkloads[qname] = "CEB"
    
df["workload"] = df.apply(lambda x: qworkloads[x["qname"]] , axis=1)
df.groupby(["workload"])["runtime"].describe(percentiles=[0.75, 0.9,0.99])

,count,mean,std,min,50%,75%,90%,99%,max
workload,,,,,,,,,
CEB,256.0,361.879533,354.644320,0.135070,249.524022,681.393749,909.000000,909.000000,909.000000
JOB,339.0,21.279154,44.066402,0.005181,5.424564,20.326218,51.011984,241.384948,415.153635


In [6]:
df["runtime"].values[0]

46.01334834098816

In [7]:
# total_times = extract_values(plan[0][0], 'Actual Total Time')
# total_times

In [8]:
import copy
def draw_plan(G):
    tmp = nx.Graph()
    for edge in G.edges():
        tmp.add_edge(edge[0], edge[1])
    pos = nx.nx_pydot.graphviz_layout(tmp, prog="dot")
    #pos = nx.nx_agraph.pygraphviz_layout(G, prog="dot")
    #print(pos)
    G = G.reverse()
    nx.draw(G, pos=pos)

In [9]:
tmp = df.sample(frac=0.1)
graphs = {}

for ri,row in tmp.iterrows():
    print(ri)
    exp = row["exp_analyze"]
    qname = row["qname"] + str(ri)

    try:
        plan = eval(str(exp))
    except Exception as e:
        print(e)
        continue


379
369
254
270
591
name 'nan' is not defined
88
48
297
542
140
343
571
142
177
535
387
506
362
name 'nan' is not defined
109
43
71
563
53
122
395
80
537
173
405
503
name 'nan' is not defined
257
24
212
368
440
144
149
312
106
192
309
49
367
383
528
513
name 'nan' is not defined
593
name 'nan' is not defined
16
358
373
428
121
124
9
416
93
250
484
23
79


In [10]:
plan

[([{'Plan': {'Node Type': 'Aggregate',
     'Strategy': 'Plain',
     'Partial Mode': 'Simple',
     'Parallel Aware': False,
     'Startup Cost': 11347.76,
     'Total Cost': 11347.77,
     'Plan Rows': 1,
     'Plan Width': 128,
     'Actual Startup Time': 12595.861,
     'Actual Total Time': 12595.899,
     'Actual Rows': 1,
     'Actual Loops': 1,
     'Plans': [{'Node Type': 'Nested Loop',
       'Parent Relationship': 'Outer',
       'Parallel Aware': False,
       'Join Type': 'Inner',
       'Startup Cost': 9.89,
       'Total Cost': 11347.75,
       'Plan Rows': 1,
       'Plan Width': 80,
       'Actual Startup Time': 540.793,
       'Actual Total Time': 12593.094,
       'Actual Rows': 1273,
       'Actual Loops': 1,
       'Inner Unique': True,
       'Join Filter': '(mi.movie_id = t.id)',
       'Rows Removed by Join Filter': 0,
       'Plans': [{'Node Type': 'Nested Loop',
         'Parent Relationship': 'Outer',
         'Parallel Aware': False,
         'Join Type': 'In